In [1]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
import requests
import logging
from datetime import datetime, timedelta
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut
from src.databaseService.Merger import Merger_AV
from src.databaseService.Parser import Parser_AV
from src.databaseService.CleanData import CleanData
from src.common.AssetDataService import AssetDataService

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

logging.getLogger().setLevel(logging.INFO)

In [2]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [3]:
ticker = "AAPL"
ts = TimeSeries(key=apiKey, output_format='pandas')

fullSharePrice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

parser = Parser_AV(
    incStatementData=incStatementData, 
    cashFlowData=cashFlowData, 
    balanceSheetData=balanceSheetData, 
    earningsData=earningsData)

financials_annually, financials_quarterly = parser.to_pandas()

financials_quarterly = CleanData.financial_fiscalDateIncongruence(financials_quarterly, daysDiscrep = 15)
financials_annually = CleanData.financial_fiscalDateIncongruence(financials_annually, daysDiscrep = 60)
financials_annually = CleanData.financial_lastRow_removeIfOutOfFiscal(financials_annually)

## Testing merge_shareprice

In [4]:
dates_init = ['2021-01-02', '2021-01-03']
_df_init = pd.DataFrame({
    'Date': dates_init,
    'Open': [100, 101],
    'High': [110, 111],
    'Low': [90, 91],
    'Close': [105, 106],
    'AdjClose': [105, 106],
    'Volume': [1000, 1100],
    'Dividends': [0, 0],
    'Splits': [1, 1]
})

def make_full_shareprice(dates, overrides=None):
    defaults = {
        '1. open': 10.0,
        '2. high': 11.0,
        '3. low': 9.0,
        '4. close': 10.5,
        '5. adjusted close': 10.5,
        '6. volume': 1000,
        '7. dividend amount': 0.0,
        '8. split coefficient': 1.0,
    }
    data = {}
    for key, val in defaults.items():
        if overrides and key in overrides:
            data[key] = overrides[key]
        else:
            data[key] = [val] * len(dates)
    idx = pd.DatetimeIndex(dates, name='date')
    return pd.DataFrame(data, index=idx)

test_Asset = AssetDataService.defaultInstance(ticker = "TEST")
# start with one annual entry and one quarterly entry, with some NaNs
test_Asset.financials_annually = pd.DataFrame([
    {"fiscalDateEnding": "2022-12-31", "reportedEPS": None, "grossProfit": 100, "totalRevenue": None,
     "ebit": 10, "ebitda": 15, "totalAssets": None, "totalCurrentLiabilities": 50,
     "totalShareholderEquity": None, "operatingCashflow": None}
])
test_Asset.financials_quarterly = pd.DataFrame([
    {"fiscalDateEnding": "2023-03-31", "reportedDate": pd.Timestamp(year=2023, month=4, day=15, hour=6), "reportedEPS": None,
     "estimatedEPS": 1.0, "surprise": 0.1, "surprisePercentage": 10, "reportTime": "AMC",
     "grossProfit": None, "totalRevenue": 200, "ebit": None, "ebitda": 20,
     "totalAssets": 500, "totalCurrentLiabilities": None,
     "totalShareholderEquity": 300, "commonStockSharesOutstanding": None,
     "operatingCashflow": 30}
])

# Cell 2: prepare new annual DataFrame with a new date and with some fields to fill
df_ann = pd.DataFrame([
    # same year but different date → should log and skip
    {"fiscalDateEnding": pd.to_datetime("2022-11-30"), "reportedEPS": 2.0, "grossProfit": 110,
     "totalRevenue": 1000, "ebit": 12, "ebitda": 18, "totalAssets": 600,
     "totalCurrentLiabilities": 60, "totalShareholderEquity": 400,
     "operatingCashflow": 50},
    # brand‐new year → should append
    {"fiscalDateEnding": pd.to_datetime("2023-12-31"), "reportedEPS": 3.0, "grossProfit": 120,
     "totalRevenue": 1100, "ebit": 14, "ebitda": 22, "totalAssets": 700,
     "totalCurrentLiabilities": 70, "totalShareholderEquity": 500,
     "operatingCashflow": 60}
])
# Cell 3: prepare new quarterly DataFrame with multiple quarters
today = datetime.now().date()
recent_q = today - timedelta(days=15)
old_q = today - timedelta(days=100)

df_quar = pd.DataFrame([
    # same quarter date as existing → should fill missing EPS and grossProfit
    {"fiscalDateEnding": pd.to_datetime("2023-03-31"), "reportedDate": pd.Timestamp(year=2023, month=4, day=15, hour=6),
     "reportedEPS": 0.5, "estimatedEPS": None, "surprise": None, "surprisePercentage": None,
     "reportTime": None, "grossProfit": 105, "totalRevenue": None, "ebit": 11,
     "ebitda": None, "totalAssets": None, "totalCurrentLiabilities": 55,
     "totalShareholderEquity": None, "commonStockSharesOutstanding": 1000,
     "operatingCashflow": None},
    # very recent quarter → should append
    {"fiscalDateEnding": pd.to_datetime(recent_q), "reportedDate": (pd.Timestamp(recent_q) + timedelta(days=10)),
     "reportedEPS": 0.8, "estimatedEPS": 0.9, "surprise": 0.1, "surprisePercentage": 12,
     "reportTime": "BMO", "grossProfit": 130, "totalRevenue": 210, "ebit": 21,
     "ebitda": 25, "totalAssets": 520, "totalCurrentLiabilities": 65,
     "totalShareholderEquity": 320, "commonStockSharesOutstanding": 1100,
     "operatingCashflow": 35},
    # old quarter, new season → should append if no same‐Q in that year
    {"fiscalDateEnding": pd.to_datetime(old_q), "reportedDate": (pd.Timestamp(old_q) + timedelta(days=10)),
     "reportedEPS": 0.6, "estimatedEPS": 0.7, "surprise": 0.05, "surprisePercentage": 8,
     "reportTime": "AMC", "grossProfit": 115, "totalRevenue": 190, "ebit": 13,
     "ebitda": 17, "totalAssets": 510, "totalCurrentLiabilities": 52,
     "totalShareholderEquity": 310, "commonStockSharesOutstanding": 1050,
     "operatingCashflow": 32}
])

## Shareprice

In [5]:
# Test 0: init
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
merger.merge_shareprice(fullSharePrice)
result = asset.shareprice
assert len(result) > 2, f"Expected mroe than 2 rows, got {len(result)}"
resDateList = list(result['Date'].apply(lambda ts: dt.strptime(ts, '%Y-%m-%d').date()))
fullShareList = list(fullSharePrice.reset_index()['date'].apply(lambda ts: ts.date()))
assert all([a in resDateList for a in fullShareList]), "Dates do not match."
print('✅ Test 0.1 passed: New rows appended correctly.')

merger.merge_shareprice(fullSharePrice)
assert asset.shareprice.equals(result), f"Expected no changes, but asset state mutated: {asset.shareprice}" 
print('✅ Test 0.2 passed: Asset unchanged when no new rows.')

asset_empty = AssetDataService.defaultInstance(ticker = "TEST")
merger_empty = Merger_AV(asset_empty)
merger_empty.merge_shareprice(fullSharePrice)
fullSharePrice.index.name = "date"
fullSharePrice = fullSharePrice.iloc[::-1]
fullSharePrice.reset_index(inplace=True)
fullSharePrice.rename(columns={
            'date': 'Date',
            '1. open': 'Open',
            '2. high': 'High',
            '3. low': 'Low',
            '4. close': 'Close',
            '5. adjusted close': 'AdjClose',
            '6. volume': 'Volume',
            '7. dividend amount': 'Dividends',
            '8. split coefficient': 'Splits'
        }, inplace=True)
fullSharePrice['Date'] = fullSharePrice['Date'].apply(lambda ts: str(ts.date()))
tmp = asset_empty.shareprice
assert asset_empty.shareprice.equals(fullSharePrice), f"Expected takeover of fullShareprice, but it changed: {asset_empty.shareprice}"
print('✅ Test 0.3 passed: Asset unchanged when no new rows.')

# Test 1: New rows
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
asset_copy = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
full = make_full_shareprice([date(2021, 1, 2), date(2021, 1, 3)])
merger.merge_shareprice(full)
# asset.shareprice should be unchanged, so asset equals asset_copy
assert not asset.shareprice.equals(asset_copy.shareprice), f"Expected changes, but asset state did not mutate: {asset.shareprice}" 
print('✅ Test 1 passed: Asset changed when no new rows.')

# Test 2: Out-of-order input
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
dates = [date(2021, 1, 5), date(2021, 1, 4)]
full = make_full_shareprice(dates)
merger.merge_shareprice(full)
res_dates = [dt.strptime(d, '%Y-%m-%d').date() for d in asset.shareprice['Date']]
assert all(d in res_dates for d in dates), "Not all new dates present after merge."
assert res_dates == sorted(res_dates), "Dates are not sorted after merge."
print('✅ Test 2 passed: Out-of-order input merged and sorted correctly.')

# Test 3: Splits and dividends changes
asset = AssetData(ticker='TEST', shareprice=_df_init.copy())
asset_copy = AssetData(ticker='TEST', shareprice=_df_init.copy())
merger = Merger_AV(asset)
chg_date = date(2021, 1, 2)
overrides = {
    '7. dividend amount': [0.5],
    '8. split coefficient': [2.0]
}
full = make_full_shareprice([chg_date], overrides)
merger.merge_shareprice(full)
assert not asset.shareprice.equals(asset_copy.shareprice), f"Expected changes, but asset state did not mutate: {asset.shareprice}" 
print('✅ Test 3 passed: Splits and dividends changes logged without new rows.')

# Test 4: String-vs-date resilience
# Introduce malformed date in existing data
_bad = _df_init.copy()
_bad.loc[0, 'Date'] = '2021/01/02'
asset = AssetData(ticker='TEST', shareprice=_bad)
merger = Merger_AV(asset)
try:
    full = make_full_shareprice([date(2021, 1, 2)])
    merger.merge_shareprice(full)
    raise AssertionError('Expected ValueError for malformed date, but none was raised.')
except ValueError:
    print('✅ Test 4 passed: Malformed date string raises ValueError.')

✅ Test 0.1 passed: New rows appended correctly.
✅ Test 0.2 passed: Asset unchanged when no new rows.
✅ Test 0.3 passed: Asset unchanged when no new rows.
✅ Test 1 passed: Asset changed when no new rows.
✅ Test 2 passed: Out-of-order input merged and sorted correctly.
✅ Test 3 passed: Splits and dividends changes logged without new rows.
✅ Test 4 passed: Malformed date string raises ValueError.


## Financials

In [ ]:
cols_ann = [
    'fiscalDateEnding','reportedEPS','grossProfit','totalRevenue','ebit','ebitda',
    'totalAssets','totalCurrentLiabilities','totalShareholderEquity',
    'operatingCashflow'
]
cols_quar = [
    'fiscalDateEnding','reportedDate','reportedEPS','estimatedEPS','surprise',
    'surprisePercentage','reportTime','grossProfit','totalRevenue','ebit',
    'ebitda','totalAssets','totalCurrentLiabilities','totalShareholderEquity',
    'commonStockSharesOutstanding','operatingCashflow'
]

#Test 0: init merge financials
asset = AssetDataService.defaultInstance()
Merger_AV(asset).merge_financials(financials_annually, financials_quarterly)
finann = financials_annually[cols_ann].copy()
finann['fiscalDateEnding'] = finann['fiscalDateEnding'].apply(lambda ts: str(ts.date()))
finquar = financials_quarterly[cols_quar].copy()
finquar['fiscalDateEnding'] = finquar['fiscalDateEnding'].apply(lambda ts: str(ts.date()))
finquar['reportedDate'] = finquar['reportedDate'].apply(
            lambda x: x.date().__str__()
        )
resann = asset.financials_annually
resquar = asset.financials_quarterly
assert asset.financials_annually.equals(finann), f"Expected take over, but not the same annual financials."
assert asset.financials_quarterly.equals(finquar), f"Expected take over, but not the same quarterly financials."
print('✅ Test 0.1 passed: New rows appended correctly.')

asset.financials_annually = asset.financials_annually.drop(index=len(asset.financials_annually)-3)
asset.financials_quarterly = asset.financials_quarterly.drop(index=len(asset.financials_quarterly)-3)
resann = asset.financials_annually
resquar = asset.financials_quarterly
assert not asset.financials_annually.equals(finann), f"Expected difference, but the same annual financials."
assert not asset.financials_quarterly.equals(finquar), f"Expected difference, but the same quarterly financials."
Merger_AV(asset).merge_financials(financials_annually, financials_quarterly)
resann = asset.financials_annually
resquar = asset.financials_quarterly
assert asset.financials_annually.equals(finann), f"Expected take over, but not the same annual financials."
assert asset.financials_quarterly.equals(finquar), f"Expected take over, but not the same quarterly financials."
print('✅ Test 0.2 passed: Disturbance corrected correctly.')

# Test 1: Check annual results
Merger_AV(test_Asset).merge_financials(df_ann, df_quar)
ann = test_Asset.financials_annually
assert "2023-12-31" in ann['fiscalDateEnding'].values, "New annual row not appended"
# original row had None reportedEPS, should remain None (we don't overwrite non‐NaN)
# but totalRevenue was None and new for 2022 was skipped (same year), so stays None
assert ann.shape[0] == 2, f"Expected 2 annual rows, got {ann.shape[0]}"
print('✅ Test 1 passed: Check annual results.')

# Test 2: Check quarterly results
q = test_Asset.financials_quarterly
dates = set(q['fiscalDateEnding'].values)
# original 2023-03-31 should still exist
assert "2023-03-31" in dates
# recent_q and old_q (formatted) should be present
for d in [recent_q, old_q]:
    assert str(d) in dates, f"Quarter {d} missing"
# the filled fields
row = q[q['fiscalDateEnding']=="2023-03-31"].iloc[0]
assert float(row['reportedEPS']) == 0.5, "reportedEPS not filled correctly"
assert float(row['grossProfit']) == 105, "grossProfit not filled correctly"
print('✅ Test 2 passed: Check quarterly results.')

✅ Test 0.1 passed: New rows appended correctly.
✅ Test 0.2 passed: Disturbance corrected correctly.
✅ Test 1 passed: Check annual results.
✅ Test 2 passed: Check quarterly results.


c:\Users\kimer\Desktop\RandomOdyssey\src\databaseService\Merger.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  existing_ann = pd.concat((existing_ann, pd.DataFrame([new], columns=existing_ann.columns)), ignore_index=True)
